In [8]:
using CSV, DataFrames
testid = "03_hybridNN";
version = "v20251125"
results_dir = joinpath(@__DIR__, "eval");

df = CSV.read(joinpath(results_dir, "$(testid)_cv.pred_$version.csv"), DataFrame; normalizenames=true);
para = CSV.read(joinpath(results_dir, "$(testid)_hyperparams_$version.csv"), DataFrame; normalizenames=true);

In [17]:
for testid in ["MultiNN", "SiNN"]
    para = CSV.read(joinpath(results_dir, "$(testid)_hyperparams_$version.csv"), DataFrame; normalizenames=true);
    
    # move best models for each fold
    for f in para.path
        src = joinpath("./output_tmp", f)
        dst = joinpath("./model", f)
        cp(src, dst; force=true)
    end
    
    # identify the best one across the fold
    # and use it as the production one later
    para_sorted = sort(para, :r2, rev=true)   # r2 越大越好
    best = para_sorted[1, :]                # 最好的一行
    best_src = joinpath("./model", best.path)
    best_dst = joinpath("model", "$(testid)_bestm.jld")
    cp(best_src, best_dst; force=true)
end


"model/03_hybridNN_bestm.jld"

In [16]:
para_sorted

Row,h,bs,lr,act,r2,mse,best_epoch,test_fold,path
,String31,Int64,Float64,String15,Float64,Float64,Int64,Int64,String
1,"(256, 128, 64, 32, 16)",512,0.001,gelu_tanh,0.401353,0.0151401,4,5,best_model_03_hybridNN_config75_fold5.jld2
2,"(512, 256, 128, 64, 32)",512,0.001,swish,0.40071,0.014832,4,4,best_model_03_hybridNN_config47_fold4.jld2
3,"(512, 256, 128, 64, 32)",512,0.001,swish,0.399178,0.0147922,5,1,best_model_03_hybridNN_config47_fold1.jld2
4,"(128, 64, 32, 16)",512,0.001,gelu_tanh,0.38384,0.01442,5,2,best_model_03_hybridNN_config156_fold2.jld2
5,"(256, 128, 64, 32, 16)",512,0.0001,gelu_tanh,0.374502,0.014289,9,3,best_model_03_hybridNN_config81_fold3.jld2
